# Hamilton matrices for spin models

Consider a spin model of the form
\begin{equation}
    H = -\sum\limits_{ij\alpha} J_{ij}^\alpha S_i^\alpha S_j^\alpha
\end{equation}
with $i,j=1,\dots, N$ and $\alpha=x,y,z$.

## a) Implementation

In [174]:
using SparseArrays
using LinearAlgebra
using Latexify
using SymPy

In [285]:
N = 3

function reset_Js(N) 
    Js = [sparse(zeros(N,N)) for α in 1:3]; # 3 N×N matrices (α = x,y,z)
    return Js
end

Js = reset_Js(N);

# initialize Js with values here
# ...

In [283]:
# convert [n]₁₀ to [n]₂ (in reversed order of notation, that is also used for |n⟩)
n_dual(n) = digits(n, base=2, pad=N)

# Function to calculate |n⟩ ↦ |l⟩ for the action of SᵢSⱼ for α = x,y
l(n,i,j) = n + (1 - 2 * n_dual(n)[i]) * 2^(i-1) + (1 - 2 * n_dual(n)[j]) * 2^(j-1)

l (generic function with 1 method)

In [284]:
# prefactor λˣ for the x-component of the interaction of spins i and j of |n⟩ 
#    -JˣᵢⱼSˣᵢSˣⱼ|n⟩ ↦ λʸ|l⟩
function x_link_prefactor(Js, n, i, j)
    α = 1 # x =̂ 1
    return -1//4 * Js[α][i,j]
end

# prefactor λʸ for the y-component of the interaction of spins i and j of |n⟩ 
#    -JʸᵢⱼSʸᵢSʸⱼ|n⟩ ↦ λʸ|l⟩
function y_link_prefactor(Js, n, i, j)
    α = 2 # y =̂ 2
    # calculate the sign, +1 for nᵢ != nⱼ, -1 for nᵢ == nⱼ
    s = - (2 * n_dual(n)[i] - 1) * (2 * n_dual(n)[j] - 1)
    return - 1//4 * s * Js[α][i,j]
end

# prefactor λᶻ for the z-component of the interaction of spins i and j of |n⟩ 
#    -JᶻᵢⱼSᶻᵢSᶻⱼ|n⟩ ↦ λᶻ|l⟩
function z_link_prefactor(Js, n, i, j)
    α = 3 # z =̂ 3
    # calculate the sign, +1 for nᵢ == nⱼ, -1 for nᵢ != nⱼ
    s = (2 * n_dual(n)[i] - 1) * (2 * n_dual(n)[j] - 1)
    return -1//4 * s * Js[α][i,j]
end

z_link_prefactor (generic function with 2 methods)

In [447]:
function calculate_hamilton_matrix(Js, N)
    H̄ = sparse(zeros(2^N, 2^N)) # empty 2ᴺ×2ᴺ matrix
    for n in 0:(2^N-1)
        # for each |n⟩ set the non-zero matrix elements
        
        # 1. loop over all components α = x,y,z 
        for α in [1,2,3]
            # 2. loop over only non-zero links in Js
            # iterate over non-zero elements by using findnz()
            for (i, j, J) in zip(findnz(Js[α])...)
                
                if α == 1 # α =̂ x
                    m = l(n, i, j) # calculate |l⟩ from |n⟩ for i,j
                    # add to the matrix element ⟨l|H|n⟩ the x-link-term
                    H̄[m+1,n+1] += x_link_prefactor(Js, n, i, j)
                elseif α == 2 # α =̂ y
                    m = l(n, i, j) # calculate |l⟩ from |n⟩ for i,j
                    # add to the matrix element ⟨l|H|n⟩ the y-link-term
                    H̄[m+1,n+1] += y_link_prefactor(Js, n, i, j)
                else
                    # α =̂ z, diagonal element
                    H̄[n+1,n+1] += z_link_prefactor(Js, n, i, j)
                end
            end
        end
    end
    return H̄
end

calculate_hamilton_matrix (generic function with 1 method)

## b) Three-site clusters
### Ising-model
$J_{ij}^\alpha = J\delta_{\alpha, z}$ and $i < j$

In [476]:
N = 3

# Ising
Js_ising = reset_Js(N)
# 1 (J) for all i<j for α =̂ z, 0 for α ≠ z
Js_ising[3] = [
    0 1 1
    0 0 1
    0 0 0
]
#println(latexify(Js_ising[3]))

H̄_ising = calculate_hamilton_matrix(Js_ising, N)

8×8 SparseMatrixCSC{Float64, Int64} with 8 stored entries:
 -0.75   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅      ⋅ 
   ⋅    0.25   ⋅     ⋅     ⋅     ⋅     ⋅      ⋅ 
   ⋅     ⋅    0.25   ⋅     ⋅     ⋅     ⋅      ⋅ 
   ⋅     ⋅     ⋅    0.25   ⋅     ⋅     ⋅      ⋅ 
   ⋅     ⋅     ⋅     ⋅    0.25   ⋅     ⋅      ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅    0.25   ⋅      ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅    0.25    ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅     ⋅    -0.75

In [485]:
#println(latexify(
Matrix(rationalize.(H̄_ising)) * sympify("J")
#))

8×8 Matrix{Sym}:
 -3*J/4    0    0    0    0    0    0       0
      0  J/4    0    0    0    0    0       0
      0    0  J/4    0    0    0    0       0
      0    0    0  J/4    0    0    0       0
      0    0    0    0  J/4    0    0       0
      0    0    0    0    0  J/4    0       0
      0    0    0    0    0    0  J/4       0
      0    0    0    0    0    0    0  -3*J/4

#### Eigenenergies

In [490]:
E_ising = eigvals(Matrix(rationalize.(H̄_ising)) * sympify("J"))
#eigvecs(Matrix(rationalize.(H̄_ising)) * sympify("J"))

2-element Vector{Sym}:
 -3*J/4
    J/4

### Isotropic Heisenberg
$J_{ij}^\alpha = J$ and $i<j$

In [492]:
N = 3

# Isotropic Heisenberg
Js_Heisenberg = reset_Js(N)
# 1 (J) for all i<j for α =̂ z, 0 for α ≠ z
Js_Heisenberg[1] = [
    0 1 1
    0 0 1
    0 0 0
]
Js_Heisenberg[2] = Js_Heisenberg[1]
Js_Heisenberg[3] = Js_Heisenberg[1]

H̄_Heisenberg = calculate_hamilton_matrix(Js_Heisenberg, N)

8×8 SparseMatrixCSC{Float64, Int64} with 32 stored entries:
 -0.75    ⋅      ⋅     0.0     ⋅     0.0    0.0     ⋅ 
   ⋅     0.25  -0.5     ⋅    -0.5     ⋅      ⋅     0.0
   ⋅    -0.5    0.25    ⋅    -0.5     ⋅      ⋅     0.0
  0.0     ⋅      ⋅     0.25    ⋅    -0.5   -0.5     ⋅ 
   ⋅    -0.5   -0.5     ⋅     0.25    ⋅      ⋅     0.0
  0.0     ⋅      ⋅    -0.5     ⋅     0.25  -0.5     ⋅ 
  0.0     ⋅      ⋅    -0.5     ⋅    -0.5    0.25    ⋅ 
   ⋅     0.0    0.0     ⋅     0.0     ⋅      ⋅    -0.75

In [524]:
#println(latexify(
Matrix(rationalize.(H̄_Heisenberg)) * sympify("J")
#))

8×8 Matrix{Sym}:
 -3*J/4     0     0     0     0     0     0       0
      0   J/4  -J/2     0  -J/2     0     0       0
      0  -J/2   J/4     0  -J/2     0     0       0
      0     0     0   J/4     0  -J/2  -J/2       0
      0  -J/2  -J/2     0   J/4     0     0       0
      0     0     0  -J/2     0   J/4  -J/2       0
      0     0     0  -J/2     0  -J/2   J/4       0
      0     0     0     0     0     0     0  -3*J/4

#### Eigenenergies

In [506]:
E_Heisenberg = eigvals(Matrix(rationalize.(H̄_Heisenberg)) * sympify("J"))
#eigvecs(Matrix(rationalize.(H̄_Heisenberg)) * sympify("J"))

8×8 Matrix{Sym}:
 1  0  0  0   0   0   0   0
 0  1  0  0  -1  -1   0   0
 0  1  0  0   1   0   0   0
 0  0  1  0   0   0  -1  -1
 0  1  0  0   0   1   0   0
 0  0  1  0   0   0   1   0
 0  0  1  0   0   0   0   1
 0  0  0  1   0   0   0   0

### model 1d
$J_{12}^x = J_{23}^y = J_{13}^z = J$ and all other $J_{ij}^\alpha = 0$ and $i<j$

In [521]:
N = 3

# Isotropic Heisenberg
Js_1d = reset_Js(N)
# 1 (J) for all (1,2,x), (2,3,y), (1,3,z)
Js_1d[1][1,2] = 1
Js_1d[2][2,3] = 1
Js_1d[3][1,3] = 1

H̄_1d = calculate_hamilton_matrix(Js_1d, N)

8×8 SparseMatrixCSC{Float64, Int64} with 24 stored entries:
 -0.25    ⋅      ⋅    -0.25    ⋅      ⋅     0.25    ⋅ 
   ⋅     0.25  -0.25    ⋅      ⋅      ⋅      ⋅     0.25
   ⋅    -0.25  -0.25    ⋅    -0.25    ⋅      ⋅      ⋅ 
 -0.25    ⋅      ⋅     0.25    ⋅    -0.25    ⋅      ⋅ 
   ⋅      ⋅    -0.25    ⋅     0.25    ⋅      ⋅    -0.25
   ⋅      ⋅      ⋅    -0.25    ⋅    -0.25  -0.25    ⋅ 
  0.25    ⋅      ⋅      ⋅      ⋅    -0.25   0.25    ⋅ 
   ⋅     0.25    ⋅      ⋅    -0.25    ⋅      ⋅    -0.25

In [525]:
#println(latexify(
Matrix(rationalize.(H̄_1d)) * sympify("J")
#))

8×8 Matrix{Sym}:
 -J/4     0     0  -J/4     0     0   J/4     0
    0   J/4  -J/4     0     0     0     0   J/4
    0  -J/4  -J/4     0  -J/4     0     0     0
 -J/4     0     0   J/4     0  -J/4     0     0
    0     0  -J/4     0   J/4     0     0  -J/4
    0     0     0  -J/4     0  -J/4  -J/4     0
  J/4     0     0     0     0  -J/4   J/4     0
    0   J/4     0     0  -J/4     0     0  -J/4

In [535]:
E_1d = eigvals(Matrix(rationalize.(H̄_1d)) * sympify("J"))
#eigvecs(Matrix(rationalize.(H̄_1d)) * sympify("J"))

2-element Vector{Sym}:
 -sqrt(3)*J/4
  sqrt(3)*J/4